### Packages to import

In [1]:
from tika import parser 
import pandas as pd 
import os
import re
import folium

### Function to convert DMS to decimal degrees

In [2]:
def decimaldegree(degree, minutes, seconds, hemisphere):
    """
    This function converts GPS coordinates in degrees, minutes, seconds 
    to decimal degrees
    """
    if hemisphere.lower() in ["w", "s", "west", "south"]:
        factor = -1.0
    elif hemisphere.lower() in ["n", "e", "north", "east"]:
        factor = 1.0
    else:
        raise ValueError("invalid hemisphere")

# check the order of operations in your code
    return factor * (float(degree) + float(minutes)/60 + float(seconds)/3600)

### Function to parse PDF for GPS coordinates
You will get a saved csv file with gps coordinates saved to your desktop and a html link
for the map will be saved in the extract folder

In [5]:
# define a function that will parse pdf, return coordinates, and return map and coordinates dataframe
def extract(pdf_path):
    """
    This function will parse the PDF into a string and from there it will extract 
    out gps coordiantes from the document and save a csv of GPS coordinates and 
    map 
    """
    # opening up PDF with tika parser
    parsed_pdf = parser.from_file(pdf_path)
    
    # saving content of PDF
    pdf_data = parsed_pdf['content'] 

    # get rid of all white space from parsed PDF
    cleaned_data = "".join(pdf_data.split())

    # pattern for gps coordinates
    pattern_1 = re.compile('''(\d{1,3}°\d{1,3}′\d{1,3}.\d{1,3}′′[A-Za-z],\d{1,3}°\d{1,3}′\d{1,3}.\d{1,3}′′[A-Za-z])''')
    pattern_2 = re.compile('''(\d{1,3}°\d{1,3}'[A-Za-z],\d{1,3}°\d{1,3}'[A-Za-z])''') 
    
    # match the pattern to the parsed data.
    gps_coords = pattern_1.findall(cleaned_data)
    
    # split list into latitude and longitude
    split_coords = []

    for elem in gps_coords:
        lst = elem.split(",")
        lst2 = [lst[0], lst[1]]
        split_coords.append(lst2)

    # convert dictionary to data frame 
    coords_df = pd.DataFrame(split_coords, columns=['Latitude', 'Longitude'])
    
    # split strings in data frame to degrees, minutes, seconds
    coords_df[['Lat_deg']] = coords_df['Latitude'].str.split("°").str[0]
    coords_df[['Lat_min']] = coords_df['Latitude'].str.split("°").str[1].str.split("′").str[0]
    coords_df[['Lat_sec']] = coords_df['Latitude'].str.split("′").str[1].str.split("′′").str[0]
    coords_df[['Lat_hem']] = coords_df['Latitude'].str.split("′′").str[1]
    coords_df[['Lon_deg']] = coords_df['Longitude'].str.split("°").str[0]
    coords_df[['Lon_min']] = coords_df['Longitude'].str.split("°").str[1].str.split("′").str[0]
    coords_df[['Lon_sec']] = coords_df['Longitude'].str.split("′").str[1].str.split("′′").str[0]
    coords_df[['Lon_hem']] = coords_df['Longitude'].str.split("′′").str[1]
    
    # apply that function along to rows, using lambda to specify the columns to use as input
    coords_df['Lat_dd'] = coords_df.apply(
        lambda row: decimaldegree(row['Lat_deg'], row['Lat_min'], row['Lat_sec'], row['Lat_hem']),
        axis=1, result_type='expand'
        )
    
    coords_df['Lon_dd'] = coords_df.apply(
        lambda row: decimaldegree(row['Lon_deg'], row['Lon_min'], row['Lon_sec'], row['Lon_hem']),
        axis=1, result_type='expand'
        )
    
    # plot the GPS points from coords_df
    # create the map
    pdf_map = folium.Map(coords_df[['Lat_dd', 'Lon_dd']].mean().values.tolist())
    
    # markers for points in map
    for lat, lon in zip(coords_df['Lat_dd'], coords_df['Lon_dd']):
        folium.Marker([lat, lon]).add_to(pdf_map)
    
    # constrain the map    
    sw = coords_df[['Lat_dd', 'Lon_dd']].min().values.tolist()
    ne = coords_df[['Lat_dd', 'Lon_dd']].max().values.tolist()
    pdf_map.fit_bounds([sw, ne]) 
    
    # display map
    display(pdf_map)
    
    return coords_df
    
    # save map
    #pdf_map.save('pdf_map.html')
    
    # define path to user root for csv file of gps coordinates can save
    #path = os.path.realpath("../pdf_gps.csv")
    
    #save the csv
    #coords_df.to_csv(path)

### This line runs the above function to parse the test pdf in the extract project folder

In [6]:
extract(os.path.realpath("../documents/MurphyRTL2017.pdf"))

,Latitude,Longitude,Lat_deg,Lat_min,Lat_sec,Lat_hem,Lon_deg,Lon_min,Lon_sec,Lon_hem,Lat_dd,Lon_dd
0,21°52′09.3′′S,43°21′16.1′′E,21,52,09.3,S,43,21,16.1,E,-21.869250,43.354472
1,23°10′28.2′′S,43°57′42.2′′E,23,10,28.2,S,43,57,42.2,E,-23.174500,43.961722
2,23°34′52.93′′S,43°49′57.24′′E,23,34,52.93,S,43,49,57.24,E,-23.581369,43.832567
3,23°29′12.79′′S,44°4′26.52′′E,23,29,12.79,S,44,4,26.52,E,-23.486886,44.074033
4,23°31′46.15′′S,44°5′31.27′′E,23,31,46.15,S,44,5,31.27,E,-23.529486,44.092019
5,23°31′23.79′′S,44°9′38.34′′E,23,31,23.79,S,44,9,38.34,E,-23.523275,44.160650
6,23°34′28.73′′S,44°19′41.53′′E,23,34,28.73,S,44,19,41.53,E,-23.574647,44.328203
7,25°10′15.07′′S,44°32′8.32′′E,25,10,15.07,S,44,32,8.32,E,-25.170853,44.535644
8,25°5′56.38′′S,44°37′3.07′′E,25,5,56.38,S,44,37,3.07,E,-25.098994,44.617519
9,25°5′11.88′′S,44°37′15.40′′E,25,5,11.88,S,44,37,15.40,E,-25.086633,44.620944
